# VER GPU

In [ ]:
!nvidia-smi
import torch

if torch.cuda.is_available():
    print("GPU disponible")
else:
    print("GPU no disponible")

# IMPORTAR JSON

In [1]:
from custom_transforms import *
from dataset import *
from generarModelo import *
from torch.nn.functional import normalize
from validar_transformaciones import *


izquierda_dir = r"C:/Users/Alejandro/TFG_ENTRENAMIENTO/JSON_FORMADOS/IZQUIERDA_NORMALIZADO"
derecha_dir = r"C:/Users/Alejandro/TFG_ENTRENAMIENTO/JSON_FORMADOS/DERECHA_NORMALIZADO"

porcentaje_entrenamiento = 0.70
porcentaje_validacion = 0.15
porcentaje_prueba = 0.15

datos_entrenamiento, datos_validacion, datos_prueba, etiquetas_entrenamiento, etiquetas_validacion, etiquetas_prueba = cargar_y_preparar_datos(izquierda_dir, derecha_dir, porcentaje_entrenamiento, porcentaje_validacion, porcentaje_prueba)

Loading data: 100%|██████████| 7694/7694 [00:19<00:00, 396.61it/s]


# COMPARAR ANTES Y DESPUÉS TRANSFORMACION

In [ ]:
#Generar transformacion TAMAÑO
datos_entrenamiento_transformados_zoom = transformacion_zoom(datos_entrenamiento[38],etiquetas_entrenamiento[38])[0]
etiquetas_entrenamiento_transformados_zoom = etiquetas_entrenamiento

#Generar transformacion TRANSLACION EJE X
datos_entrenamiento_transformados_translacionX = desplazar_posesX(datos_entrenamiento[5],etiquetas_entrenamiento[5])[0]
etiquetas_entrenamiento_transformados_translacionX = etiquetas_entrenamiento
#
##Generar transformacion TRANSLACION EJE Y
datos_entrenamiento_transformados_translacionY = desplazar_posesY(datos_entrenamiento[5],etiquetas_entrenamiento[5])[0]
etiquetas_entrenamiento_transformados_translacionY = etiquetas_entrenamiento
#
##Generar transformacion FLIP
datos_entrenamiento_transformados_flip = flip_poses(datos_entrenamiento[5],etiquetas_entrenamiento[5])[0]
etiquetas_entrenamiento_transformados_flip = invertir_etiquetas(etiquetas_entrenamiento)


# VALIDAR TRANSFORMACIONES

In [ ]:
# ZOOM
validacionZoom(datos_entrenamiento,etiquetas_entrenamiento)

In [ ]:
# TRANSLACION X
validacionX(datos_entrenamiento,etiquetas_entrenamiento)

In [ ]:
# TRANSLACION Y
validacionX(datos_entrenamiento,etiquetas_entrenamiento)

In [ ]:
# FLIP
validacionFlip(datos_entrenamiento,etiquetas_entrenamiento)

# VER TENSORES TRANSFORMADOS

In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image


unir_gifs(gif(datos_entrenamiento[38]), gif(datos_entrenamiento_transformados_zoom), "resultadoZoom.gif")

Image("resultadoZoom.gif")


In [ ]:
print(datos_entrenamiento[5])
print((datos_entrenamiento[5])-(datos_entrenamiento_transformados_zoom))

In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image

unir_gifs(gif(datos_entrenamiento[5]), gif(datos_entrenamiento_transformados_translacionX), "resultadoX.gif")

Image("resultadoX.gif")

In [ ]:
from gifComparaTransformado import *
from generar_gif import gif
from IPython.display import Image

unir_gifs(gif(datos_entrenamiento[5]), gif(datos_entrenamiento_transformados_translacionY), "resultadoY.gif")

Image("resultadoY.gif")

In [ ]:
from gifComparaTransformado import *
from generar_gif import *

from IPython.display import Image


unir_gifs(gif(datos_entrenamiento[5]), gif(datos_entrenamiento_transformados_flip), "resultadoFlip.gif")

Image("resultadoFlip.gif")

# VER LOGS

In [ ]:
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs/fino --port 6457

# ENTRENAR OPTUNA

In [2]:
# Luego, puedes utilizar los DataLoader en las funciones train_with_optuna y train_final_model
from CustomDataset import *
study = train_with_optuna(datos_entrenamiento, etiquetas_entrenamiento, datos_validacion, etiquetas_validacion, num_trials=1, num_epochs=5, batch_size=128)


# Luego, entrenamos el modelo final con los mejores parámetros encontrados por Optuna


[I 2024-04-15 21:54:23,691] A new study created in memory with name: no-name-95e7565d-a688-47b8-822c-556b53364f12
Trial 0/ 1: 100%|██████████| 5/5 [01:11<00:00, 14.21s/epoch]
[I 2024-04-15 21:55:36,073] Trial 0 finished with value: 0.4982668977469671 and parameters: {'learning_rate': 2.1414769169443658e-06, 'lstm_units': 1183, 'dense_units': 1118, 'dropout_rate': 0.20774258752617092}. Best is trial 0 with value: 0.4982668977469671.


# ENTRENAR FINAL

In [ ]:
print(study.best_trial.params)

In [ ]:
model, accuracy = train_final_model(datos_entrenamiento, etiquetas_entrenamiento, datos_validacion, etiquetas_validacion,  study, num_epochs=100, batch_size=128)


Final Training:   1%|          | 1/100 [00:14<23:43, 14.38s/epoch]

Epoch 1/100: Train Loss: 0.6932, Train Acc: 0.5043, Val Loss: 0.6930, Val Acc: 0.5130


Final Training:   2%|▏         | 2/100 [00:28<23:23, 14.33s/epoch]

Epoch 2/100: Train Loss: 0.6934, Train Acc: 0.4888, Val Loss: 0.6933, Val Acc: 0.4944


Final Training:   3%|▎         | 3/100 [00:43<23:13, 14.37s/epoch]

...

Epoch 65/100: Train Loss: 0.6932, Train Acc: 0.5016, Val Loss: 0.6931, Val Acc: 0.5039


Final Training:  66%|██████▌   | 66/100 [16:26<08:29, 14.97s/epoch]

Epoch 66/100: Train Loss: 0.6932, Train Acc: 0.5027, Val Loss: 0.6931, Val Acc: 0.4983


Final Training:  67%|██████▋   | 67/100 [16:41<08:17, 15.07s/epoch]

Epoch 67/100: Train Loss: 0.6932, Train Acc: 0.4971, Val Loss: 0.6931, Val Acc: 0.5069
